<br/>
<span style="font-size: 3em;"> 🚢🐼🌳 Engenharia de Features com Pandas</span>

Neste `notebook` vamos estudar a biblioteca `pandas`, usando um conjunto de dados dos passageiros do **RMS Titanic**. Por último, tentaremos extrair informações dos dados a partir de árvores de decisão.

![Pandas](dados/944693_1_1029_panda_diplomacy_standard.jpg)


## O Que é `pandas`?

> ### Python Data Analysis Library
> Um conjunto de ferramentas para análise de dados em `Python`.

# 1. DataFrames
A estrutura de dados mais utilizada em `pandas` é o [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), que armazena os dados na forma de uma tabela. Os dados são indexados pelas colunas (*columns*) e pelas linhas (*indexes*).

Os `DataFrames` também possuem diversos métodos para computar estatísticas e processar os dados.

Existem várias maneiras de declarar um `DataFrame`. Uma forma comum é passar um dicionário (`dict`) no qual as chaves são as colunas e os valores são listas das linhas:

In [ ]:
import pandas as pd
from sklearn import metrics, tree

# Podem ignorar os imports abaixo
import os
if os.name == "nt":
    os.environ["PATH"] += os.pathsep + "C:/Program Files (x86)/Graphviz2.38/bin/"

import graphviz
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
PALETA = [item["color"] for item in list(mpl.rcParams['axes.prop_cycle'])]

In [ ]:
dados = {
    "bamboo": [0, 10, 20],
    "taquara": [5, 15, 25],
    "grama": [-1, -2, None],
}
df_panda = pd.DataFrame(dados)
df_panda  # Somente para exibir a tabela

O valor de retorno da última linha de um bloco de código é exibido no notebook, por isso a tabela acima é exibida.

Também é possível passar apenas os valores e declarar as colunas ou linhas no construtor do `DataFrame`:

In [ ]:
dados = [[0, 5, -1], [10, 15, -2], [20, 25, None]]
pd.DataFrame(dados, columns=["bamboo", "taquara", "grama"])

Provavelmente as formas mais utilizadas para carregar um `DataFrame` são aquelas que carregam dados em disco. Entre os formatos suportados estão:
1. CSV
1. Excel
1. HDFS
1. JSON
1. Pickle
1. Parquet
1. SQL

Temos um arquivo `csv` de exemplo em `dados/train.csv`. As primeiras linhas do arquivo são:

In [ ]:
ARQUIVO_TRAIN = "dados/train.csv"

with open(ARQUIVO_TRAIN, "r") as fp:
    dados = fp.read()
    print("O arquivo: {0} contém {1} linhas\n".format(ARQUIVO_TRAIN, len(dados.split("\n"))-1))
    print("\n".join(dados.split("\n")[:5]))

Para abrir um arquivo `csv` basta invocar a função `read_csv` do `pandas`. Os métodos `head` e `tail` permitem investigar o começo e o fim do `DataFrame`.

In [ ]:
df = pd.read_csv(ARQUIVO_TRAIN)
df.head()

In [ ]:
df.tail()

Vamos usar o método `head` para mostrar apenas um pedaço dos `DataFrames` daqui em diante.

----
## Exercícios

### E1.1:
Declare um `DataFrame` com o nome `df_teste` abrindo o arquivo `dados/test.csv` usando a função `read_csv`.

In [ ]:
# Preencher a linha abaixo
df_teste =

# A linha abaixo serve somente para mostrar o dataframe 
df_teste.head()

### E1.2:
Declare um `DataFrame` com o nome de `df_filmes` contendo os seguintes dados:

|Filme|Ano|País|Amor|
|---|---|---|---|
|Saved from The Titanic|1912|Estados Unidos|False|
|In Nacht und Eis|1912|Alemanha|False|
|Atlantik|1929|Inglaterra/Alemanha|False|
|Titanic|1943|Alemanha|False|
|Titanic|1953|Estados Unidos|False
|A Night to Remember|1958|Inglaterra|False|
|Titanica|1992|Estados Unidos/Russia/Canadá|False|
|Titanic|1997|Estados Unidos|True|
|La leggenda del Titanic|1999|Itália|False|
|Ghosts of the Abyss|2003|Estados Unidos|False|
|[Titanic II](https://en.wikipedia.org/wiki/Titanic_II_(film))|2010|Estados Unidos|False|

In [ ]:
### Responder nas linhas abaixo











# A linha abaixo serve somente para mostrar o dataframe 
df_filmes

# 2. ... near, far, wherever you are

<img src="dados/titanic-movie-promo-stills-wallpaper-4.jpg" width=800>

Vamos entender um pouco do conjunto de dados que acabamos de carregar. 

Pra quem nunca viu o filme, o navio **RMS Titanic**, considerado inaufragável, partiu no dia 10 de abril de 1912, de Southampton, com destino à Nova York, fazendo paradas em Cherbourg e Queenstown. Às 23:40 da noite do dia 14 de abril, o navio colidiu com um iceberg, danificando o lado de estibordo da embarcação e iniciando a inundação de 5 compartimentos, o que inevitávelmente causaria seu naufrágio.

![Mapa Titanic](dados/Titanic_voyage_map.png)

O navio se parte ao meio às 2:20 da madrugada com mais de 1000 passageiros à bordo. Das estimadas 2224 pessoas à bordo, cerca de 705 sobreviveram. O Titanic estava a dois dias de viagem de seu destino.

### [spoilers]
> Nesse meio tempo, Rose, que está noiva, conhece Jack, rola um arrocha, ela larga o boy lixo pra ficar com Jack, o navio afunda e o casal também. Rose consegue subir numa porta à deriva e Jack não tem competência suficiente pra subir na porta e acaba morrendo.

<img src="dados/titanic_door.jpg" width=600>

## Conjunto de dados

Cada linha do conjunto de dados representa um passageiro. As colunas são as seguintes:

1. PassengerId: Id único do passageiro
1. Survived: (1) se o passageiro sobreviveu ao naufrágio, (0) caso contrário
1. Pclass: Classe do passageiro (1ª, 2ª ou 3ª)
1. Name: Nome do passageiro
1. Sex: Sexo do passageiro
1. SibSp: Número de irmãos ou esposo(a)s
1. Parch: Número de pais ou filhos
1. Ticket: Número do ticket
1. Fare: Tarifa da passagem
1. Cabin: Número da cabine
1. Embarked: Cidade em que o passageiro embarcou (C - Cherbourg, S - Southampton, Q - Queenstown)

Os dados são da competição de _Machine Learning_ do [Kaggle](https://www.kaggle.com/c/titanic)

## Definição do problema

O objetivo da competição do [Kaggle](https://www.kaggle.com), no conjunto de dados do **Titanic**, é de gerar um modelo capaz de determinar quais passageiros sobreviveram ao naufrágio. 

O _Kaggle_ fornece um conjunto de dados dividido em duas partes, a primeira consiste em uma lista com 891 passageiros e suas características (features), como vimos anteriormente. Chamaremos essa fatia dos dados de _conjunto de treino_ . A segunda parte é uma lista com 418 passageiros com as mesmas características, exceto pela coluna `Survived`, que somente o _Kaggle_ tem acesso. Chamaremos essa outra parte dos dados de _conjunto de testes_ .

> O objetivo, então, é gerar a coluna `Survived` para a segunda parte do conjunto de dados e enviar os dados para o _Kaggle_ , que calcula a quantidade de acertos.

Podemos explorar os dados de teste por meio do `DataFrame` `df_teste`:

In [ ]:
ARQUIVO_TESTE = "dados/test.csv"
df_teste = pd.read_csv(ARQUIVO_TESTE)
df_teste.head()

In [ ]:
# Fique à vontade para explorar o DataFrame


Antes de <span style="font-size: 1.5em">mergulhar 🤦</span> no problema, vamos entender um pouco mais sobre `DataFrames`. Vamos pré-processar os dados e isso será útil para entender as operações que faremos posteriormente.

## 2.1. Indexação e Seleção de dados
O método `loc` é utilizado para selecionar dados no `DataFrame`. Esse método implementa o `__getitem__`, ou seja, ao contrário de `__call__`, que usa parênteses `()` para chamar uma função, esse método é invocado por colchetes `[]`.

In [ ]:
df.head()

Para ler uma linha do `DataFrame`, basta passar um único índice para o método `loc`:

In [ ]:
df.loc[0]

Também é possível passar um intervalo para o método:

In [ ]:
df.loc[0:2]

Ou então uma lista de índices:

In [ ]:
df.loc[[0, 3, 5]]

Para um `DataFrame` também é possível selecionar colunas com o método `loc` usando o segundo argumento:

In [ ]:
df.loc[:, ["Name"]].head()

Outra forma de indexação é feita diretamente sobre o objeto do `DataFrame` usando colchetes `[]` (`__getitem__`), selecionando as colunas. Chamaremos este modo de **indexação básica**.

In [ ]:
df["Name"].head()

In [ ]:
df[["Name"]].head()

Qual é a diferença entre as duas últimas chamadas?

```python 
df["name"].head()
# e #
df[["name"]].head()
```

> ### Pandas Series
É importante ressaltar que existe outra estrutura de dados importante em `pandas` que se chama [`Series`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html). Esta estrutura representa uma sequência de dados de apenas uma dimensão, enquanto o `DataFrame` tem duas dimensões. Análogamente à algebra linear, as `Series` seriam equivalentes às **listas**, enquanto os `DataFrame`s seriam equivalentes às **matrizes**. Muitos métodos dessas duas estruturas são idênticos.
>
> As linhas e colunas de um `DataFrame` são representadas por uma `Series`.
>
> Uma instância de uma `Series` é feita com a classe `pd.Series`.

In [ ]:
pd.Series(["🚢", "🧊", "🕳️", "💀"])

---
## Exercícios

### E2.1.1
Qual é o nome do passageiro na linha `456`?

### E2.1.2
Selecione a coluna `Cabin` dos passageiros das linhas `571` até `581`

## 2.2. Indexação Booleana

Algumas operações em `pandas` retornam séries *booleanas*, que podem ser usadas para selecionar (filtrar) os dados. As formas mais comuns de filtragem envolvem a utilização dos operadores de comparação do `python`.

Para selecionar apenas os passageiros da primeira classe, primeiramente é preciso constrir uma `Series` booleana:

In [ ]:
(df["Pclass"] == 1).head(20)

Com a `Series` é possível usar a **indexação básica** para filtrar os dados:

In [ ]:
df[df["Pclass"] == 1].head()

Também é possível usar operadores lógicos e criar filtros mais complexos. Para selecionarmos os passageiros do sexo feminino e que embarcaram em *Cherbourg*, podemos fazer o seguinte:

* Primeiro criamos um filtro para o sexo feminino

In [ ]:
filtro_sexo_f = df["Sex"] == "female"
filtro_sexo_f.head(10)

* Criamos outro filtro para os passageiros que embarcaram em _Cherbourg_

In [ ]:
filtro_embarque_c = df["Embarked"] == "C"
filtro_embarque_c.head(10)

* Usamos o operador _and_ (`&`) para criar um terceiro filtro de ambas as condições

In [ ]:
filtro_sexo_f_e_embarque_c = filtro_sexo_f & filtro_embarque_c
filtro_sexo_f_e_embarque_c.head(10)

In [ ]:
df[filtro_sexo_f_e_embarque_c].head()

---
## Exercícios
### E2.2.1 
Selecione apenas os passageiros com mais de 50 anos. Use o método `head` para mostrar somente o começo do `DataFrame`:

### E2.2.2 
Selecione os passageiros que pagaram uma tarifa menor que £10 e que sobreviveram. Lembre-se do método `head`.

## 2.3. Atribuição de valores
Para atribuir um valor à uma célula é preciso usar o método `loc`:

In [ ]:
df_panda

In [ ]:
df_panda.loc[2, "grama"] = -3
df_panda

Para atribuir valores a uma coluna, basta usar a indexação direta:

In [ ]:
df_panda["floresta"] = "densa"
df_panda

A atribuição por coluna também pode receber uma série:

In [ ]:
df_panda["bamboo"] >= 20

In [ ]:
df_panda["muito bamboo"] = df_panda["bamboo"] >= 20
df_panda

Por último, para adicionar linhas ao `DataFrame`, basta usar o método `append`:

In [ ]:
df_panda = df_panda.append({
    "bamboo": 30,
    "taquara": 35,
    "muito bamboo": True
}, ignore_index=True)
df_panda

---
## Exercícios

### E2.3.1
Preencha a célula da linha `3` e coluna `grama` com o valor `-4` em `df_panda`:

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe 
df_panda

### E2.3.2
Adicione a seguinte linha ao `DataFrame` `df_panda`: 
```python
{
    "bamboo":40, 
    "taquara": 45, 
    "grama": -5, 
    "floresta": "rala", 
    "muito bamboo": True
}```

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe 
df_panda

### E2.3.3
Adicione uma coluna ao `DataFrame` `df_panda` chamada `kung fu` com o valor `True`

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe 
df_panda

## 2.4. Dados faltantes
Não há garantias de que as tabelas com as quais trabalhamos estejam completas. Decidir o que fazer com os dados faltantes é algo que deve ser estudado caso a caso. 
O método `isnull` retorna um `DataFrame` booleano no qual os valores indicam a falta de dados.

In [ ]:
df.isnull().head()

Como muitos métodos dos `DataFrame`s retornam outro `DataFrame`, podemos encadenar métodos. Podemos, por exemplo, verificar quais linhas possuem algum dado faltante, encadenando o método `any`, que verifica se **algum** elemento é **verdadeiro**.

In [ ]:
df.isnull().any(axis=1).head(20)

In [ ]:
null_cabin = df["Cabin"].isnull().sum()
"{0} dados faltantes na coluna 'Cabin' ({1:.3}% do total)".format(null_cabin, 100*null_cabin/len(df))

> O argumento `axis=1` serve para indicar que queremos comparar os elementos em uma linha. Por padrão,o método usa `axis=0`, ou seja, compara os valores em uma coluna.

> #### O método `all` verifica se **todos** os elementos são **verdadeiros**.

### 2.4.1. Eliminando colunas
Uma opção para lidar com colunas contendo poucos dados é apenas eliminá-las com o método `drop`. Os métodos de um `DataFrame` retornam uma cópia dele, portanto, podemos substituir o `DataFrame` antigo (`df`) pela cópia após a aplicação do método usando uma atribuição:

In [ ]:
df = df.drop(columns="Cabin")
df.head()

### 2.4.2. Eliminando linhas
Outra opção é remover linhas. Dessa vez é interessante que existam poucas linhas com dados faltantes de determinada coluna.

In [ ]:
df.isnull().head(6)

Aproveitamos aqui para introduzir outra computação. O método `sum` soma os os valores em uma coluna (`axis=0` por padrão). Somando os valores `True`, equivalentes a `1`, oposto de `False` $\equiv$ `0`, temos a contagem das linhas vazias:

In [ ]:
df.isnull().sum()

Como a coluna `Embarked` possui apenas 2 linhas vazias, podemos descartá-la sem perder muitos dados.

In [ ]:
df[df["Embarked"].isnull()]

In [ ]:
df.loc[59:63]

Para descartar esses dados podemos usar o método `drop` passando `index` como argumento:

In [ ]:
df.drop(index=[61, 829]).loc[59:63]

Remover `index` (índices) deixa o `DataFrame` com lacunas em seus índices. Caso o índice não seja relevante para o problema, é possível recriá-lo com o método `reset_index`:

In [ ]:
df.drop(index=[61, 829]).reset_index(drop=True).loc[59:63]

As vezes é preferível filtrar o `DataFrame` ao invés de usar o método `drop`. Podemos usar o método `notnull` ou inverter o método `isnull` com o operador `~` (not):

In [ ]:
df["Embarked"].notnull()[59:63]

In [ ]:
~df["Embarked"].isnull()[59:63]

Comparando as duas formas de determinar itens não nulos, verificamos que ambas são equivalentes:

In [ ]:
(~df["Embarked"].isnull() == df["Embarked"].notnull()).all()

In [ ]:
df = df[df["Embarked"].notnull()].reset_index(drop=True)
df.loc[59:63]

### 2.4.3. Preenchendo valores vazios (Imputação de dados)
Até agora lidamos com dados faltantes apenas eliminando tais elementos. Entretanto, a coluna `Age` (Idade) possui cerca de $1/5$ de seus valores faltantes. Seria um grande desperdício deletar essa coluna ou as linhas em que essa coluna está vazia.

In [ ]:
"A coluna 'Age' tem {:.2f}% de dados faltantes".format(df["Age"].isnull().sum()/len(df)*100)

Podemos, então, utilizar outra técnica, a *imputação de dados*. Com ela, criamos valores para os dados faltantes. Existem métodos elegantes para criar esses dados; neste notebook, vamos usar os mais simples: `média`, `mediana` e `moda`. Essas três operações são estatísticas que podemos calcular facilmente com o `DataFrame`:

In [ ]:
ax = df["Age"].hist(bins=20, figsize=(8, 6))
ax.set_title("Distribuição das idades dos passageiros")
ax.set_xlabel("Idade")
ax.set_ylabel("Frequência")

A _média_ corresponde à soma dos valores dividida pela quantidade de elementos:

ex: A média de **28, 29, 29, 30, 31, 31** é: $(28+29+29+30+31+31)/6 = 29.66...$

In [ ]:
df["Age"].mean()

A _mediana_ é o elemento central numa lista ordenada dos valores

ex: **1, 1, ... 27, 28, <span style="color: red">28</span>, 28, 29, ... 80, 81**

In [ ]:
df["Age"].median()

A _moda_ representa o valor que mais se repete dentre os elementos.

ex: **20, 21, 22, <span style="color: red">24, 24, 24,</span> 27, 28**. O número **24** é que mais se repete.

In [ ]:
df["Age"].mode()[0]

Usando o método `fillna`, podemos preencher os valores faltantes da coluna `Age`. Vamos preencher as linhas faltantes com o valor da `mediana`.

In [ ]:
df["Age"] = df["Age"].fillna(df["Age"].median())
ax = df["Age"].hist(bins=20, figsize=(8, 6))
ax.set_title("Distribuição das idades dos passageiros")
ax.set_xlabel("Idade")
ax.set_ylabel("Frequência")

Vemos que a distribuição das idades fica bastante alterada, mas pelo menos temos uma tabela com todos os valores preenchidos.

In [ ]:
df.info()

---
## Exercícios

# SE PREPAREM!

### E2.4.1
Qual é a média do valor da passagem paga pelos passageiros da terceira classe? E da primeira classe?

### E2.4.2
Quantos passageiros que estão no `DataFrame` `df` sobreviveram ao naufrágio?

### E2.4.3
Quais são as taxas de sobrevivência dos passageiros de cada uma das três classes?

### E2.4.4
Qual é a taxa de sobrevivência dos passageiros de até 16 anos? E com 16 ou mais?

### E2.4.5
Qual é a taxa de sobrevivência dos passageiros do sexo feminino? E do sexo masculino?

---
<span style="font-size: 2em">Phew!</span> Já cobrimos muita coisa:
* DataFrames
* Series
* Indexação e Seleção de Dados
* Indexação Booleana
* Atribuição de Valores
* Métodos do DataFrame
* Dados faltantes

Também pudemos tirar algumas conclusões sobre os dados do naufrágio do **Titanic**:

1. Passageiros da primeira classe têm uma taxa de sobrevivência maior:
2. Passageiros com menos de 16 anos têm uma taxa de sobrevivência maior
3. Passageiros do sexo feminino têm uma taxa de sobrevivência maior
4. Quem morre é homem, adulto e pobre.

Confira se seus cálculos dos últimos três exercícios estão corretos abaixo:

In [ ]:
filtros = [
    ("1ª Classe", df["Pclass"] == 1, PALETA[0]),
    ("2ª Classe", df["Pclass"] == 2, PALETA[0]),
    ("3ª Classe", df["Pclass"] == 3, PALETA[0]),
    ("Menor de 16 anos", df["Age"] < 16, PALETA[1]),
    ("Maior de 16 anos", df["Age"] >= 16, PALETA[1]),
    ("Sexo feminino", df["Sex"] == "female", PALETA[2]),
    ("Sexo masculino", df["Sex"] == "male", PALETA[2]),
]
eixo_x = []
eixo_y = []
paleta = []
for categoria, filtro, cor in filtros:
    eixo_x.append(categoria)
    eixo_y.append(df[filtro]["Survived"].sum()/len(df[filtro]))
    paleta.append(cor)

fig, ax = plt.subplots(figsize=(15, 6))
ax.set_ylim(0, 1)
ax.set_title("Taxas de sobrevivência")
rects = ax.bar(
    eixo_x, eixo_y,
    color=paleta,
)
for rect in rects:
    height = rect.get_height()
    ax.annotate("{:.3f}".format(float(height)),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                ha="center", va='bottom')

# 3. Árvores à bordo!
![Árvore](dados/df3f9f4bbba80a7547023b2823e6f0eb.jpg)


Para modelar os dados usaremos **Árvores de decisão**, um método de aprendizado de máquina que se utiliza de um grafo em formato de árvore binária para tomar as decisões:

![arvore engenharia](dados/tumblr_lui1cu0BzO1qe69yqo1_500.jpg)

A árvore é composta por nós, que redirecionam para o próximo ramo da árvore, e por nós-folha, que são a decisão ou a classificação. No problema desse dataset temos apenas duas categorias para as folhas, ou o passageiro sobreviveu (**1**), ou o passageiro morreu (**0**).

Vamos usar a biblioteca [scikit-learn](https://scikit-learn.org/stable/) que implementa o modelo numa interface muito simples.

## 3.1. Preparação dos dados
O modelo é capaz de lidar apenas com dados numéricos, então é preciso realizar um pré-processamento. 
* Dados categóricos com apenas duas classes podem ser convertidos para uma coluna binária
* Dados categóricos com mais de duas classes precisam ser convertidos para colunas diferentes (one hot encoding)
* Dados numéricos podem permanecer como estão

No momento vamos nos ater somente às colunas numéricas:


In [ ]:
X = df[["Age", "SibSp", "Parch", "Fare"]]
X.head()

A saída da árvore de decisão já está num formato numérico, então não precisamos alterá-la.

In [ ]:
Y = df["Survived"]
Y.head(10)

## 3.2. Avaliação do modelo
A métrica que utilizaremos para avaliar o modelo é a **acurácia**. Esta métrica representa a taxa de acertos do modelo, por exemplo: Se pegarmos os 5 primeiros passageiros:

In [ ]:
df.head()

Este subconjunto dos dados contém 3 passageiros que sobreviveram ao desastre e 2 que morreram. 

Vamos criar um modelo simples, considerando que todos os passageiros sobreviveram. Como calculamos a acurácia para esse subconjunto?
* Primeiro vamos criar um `DataFrame` contendo duas colunas, a coluna com os dados verdadeiros (coluna `Survived`) e outra com a predição do modelo:

In [ ]:
df["predicao simples"] = 1  # Consideramos que todos os passageiros sobreviveram
df[["Survived", "predicao simples"]].head()

* Em seguida criamos uma terceira coluna que compara a coluna `Survived` à coluna `predicao simples`:

In [ ]:
df["acertos predicao simples"] = df["Survived"] == df["predicao simples"]
df[["Survived", "predicao simples", "acertos predicao simples"]].head()

* Por último, determinamos a razão entre a quantidade de acertos e o tamanho do conjunto de dados. Para essas 5 amostras temos o seguinte:
  * Acertos: 3
  * Tamanho do conjunto de dados: 5
  * Acurácia: 3/5 = 0.6 (60%)

In [ ]:
acc = df["acertos predicao simples"].head().sum()/len(df.head())
"Acurácia 5 linhas: {}%".format(100*acc)

O `sklearn` possui uma função que calcula a acurácia, precisamos apenas passar a coluna `verdade` e a coluna `predição`:

In [ ]:
acc = metrics.accuracy_score(
    df["Survived"].head(), 
    df["predicao simples"].head()
)
"Acurácia 5 linhas: {}%".format(100*acc)

A acurácia para todo o conjunto de dados é:

In [ ]:
acc_simples = metrics.accuracy_score(
    df["Survived"], 
    df["predicao simples"]
)
"Acurácia: {}%".format(100*acc_simples)

## 3.3.  Árvores de decisão
Vamos jogar os dados na a árvore de decisão: 
1. Primeiro instanciamos a classe `DecisionTreeClassifier`
2. Invocamos o método `fit` para criar a árvore
3. Para realizar predições sobre a árvore, basta usar o método `predict`

In [ ]:
## Preparação dos dados
X1 = df[["Age", "SibSp", "Parch", "Fare"]]
Y1 = df["Survived"]

# 1. Instanciando o modelo
modelo_arvore1 = tree.DecisionTreeClassifier(max_depth=4, min_samples_leaf=5)

# 2. Treinando o modelo
modelo_arvore1.fit(X1, Y1)

# 3. Gerando as predições
df["predicao arvore 1"] = modelo_arvore1.predict(X1)
df[["Survived", "predicao arvore 1"]].head(10)

In [ ]:
acc_arvore1 = metrics.accuracy_score(
    df["Survived"], 
    df["predicao arvore 1"]
)
"Acurácia: {}%".format(100*acc_arvore1)

Conseguimos modelar corretamente 71.1% dos dados! Muito melhor que os 38.2% do _modelo simples_.

O _scikit-learn_ fornece uma interface simples para visualizar as árvores de decisão, com essa visualização podemos tirar algumas conclusões:

In [ ]:
def plot_tree(model, columns, classes):
    dot_data = tree.export_graphviz(
        model,
        out_file=None,
        feature_names=columns,
        class_names=classes,
        filled=True, rounded=True
    )
    graph = graphviz.Source(dot_data)
    return graph
classes = ["morreu", "sobreviveu"]
plot_tree(modelo_arvore1, X1.columns, classes)

Os nós mais coloridos representam os casos dos quais o modelo tem mais certeza na predição. Podemos concluir:
* Os passageiros que pagaram barato nas tarifas (`Fare` $<=$ 10.481) morreram com 80% $(272\div(272+67))$ de probabilidade, especialmente os mais velhos (`Age` $>$ _32.5_ ), com 94% $(58\div(58+4))$ de probabilidade.
* Passageiros que pagaram caro nas tarifas (`Fare` $>$ _74.375_ ) tiveram mais chances de sobreviver (24.2% de mortalidade).

---
## Exercícios
### E3.1.
Vamos criar outro modelo, a predição será sorteada aleatóriamente. Primeiro precisamos gerar uma lista com _889_ números aleatórios. Use o método `random.random` para gerar números aleatórios:

Dica: método [`append`](https://docs.python.org/3.7/tutorial/datastructures.html) para `lista`

In [ ]:
import random
random.seed(2019)

lista_randomica = []
for i in range(889):
    # Complete a linha abaixo
    

### E3.2.
Crie uma `Series` da lista randômica chamada `sr_randomica`:

### E3.3.
Converta a `Series` `sr_randomica` uma série _boleana_ indicando quais numeros são maiores que 0.5. Nomeie essa `Series` como `pred_randomica`. Essa é a série das predições do modelo:

### E3.4.
Converta a `Series` `pred_randomica` para o tipo inteiro usando o método [`astype`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html) passando [`int`](https://docs.python.org/3/library/functions.html#int) como parâmetro:

### E3.5.
Calcule a acurácia do modelo randômico e a armazene na variável `acc_randomica`:

In [ ]:
acc_randomica = 

# A linha abaixo serve somente para mostrar o valor
"Acurácia: {}%".format(100*acc_randomica)

47.9% de acurácia com um modelo randômico!

_Será que conseguimos melhorar ainda mais?_
# 4. Vamos para a casa de máquinas!
![casa de máquinas](dados/media_239607.jpg)
<br/>
Um jeito de tentar melhorar a acurácia é extraindo mais informações do conjunto de dados. Realizar operações em conjuntos de dados para extrair mais informações é conhecido como *Engenharia de Features*.

Vamos iniciar investigando os nomes dos passageiros:

In [ ]:
df[["Name"]].head(10)

É possível extrair alguma informação relevante do nome dos passageiros?

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.
# CLARO!

## 4.1. Manipulação de Strings

Podemos determinar a hierarquia <span style="font-size: 1.5em">🤮</span> entre os passageiros analisando o prefixo (titulo) atrelado aos nomes. Primeiro vamos procurar pelos prefixos usando [Expressões Regulares](https://pt.wikipedia.org/wiki/Express%C3%A3o_regular), para tanto usamos um dos métodos de [manipulação de strings](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#string-handling) do `pandas`, o método `extract`:

In [ ]:
expressao_titulo = " ([^. ]+\.)"
df["Prefix"] = df["Name"].str.extract(expressao_titulo)
df[["Prefix", "Name"]].head()

Podemos contar a quantidade de cada título com o método para `Series` `value_counts`:

In [ ]:
df["Prefix"].value_counts()

In [ ]:
df[df["Name"].str.contains("Jonkheer\.")]

Jonkheer é o mais baixo título de nobreza Holandês que vem de _jonk_ (jovem, young) e _here_ (senhor, mestre).
<img src="dados/a184e964efa3c63a88549d14cca0fd06.jpg" width=600>
<center><i>Jonkheer Reuchlin e sua esposa Agatha Maria Elink Schuurman</i></center>

Mas o que podemos fazer com essas informações? Podemos criar uma nova coluna nos nossos `DataFrames` para indicar a titularidade de cada passageiro. Vamos categorizar os títulos em:
1. Oficiais
  * Capt.
  * Col.
  * Major.
  * Dr.
  * Rev. (Reverendo)
2. Realeza
  * Jonkheer.
  * Don.
  * Sir.
  * Countess.
  * Lady.
3. Mr.
4. Mrs.
  * Mrs. (Mistress, geralmente a esposa que trocou o sobrenome)
  * Ms. (Miss, não indica o estado civil)
  * Mme. (Madame)
  * Dona (Somente no conjunto de dados de teste)
5. Miss./Mlle. (Miss/Mademoiselle, mulheres solteiras)
6. Master (Homens jovens)

Substituindo os títulos antigos, temos a seguinte contagem:

In [ ]:
titulos = {
    "Capt.": "Officer",
    "Col.": "Officer",
    "Major.": "Officer",
    "Dr.": "Officer",
    "Rev.": "Officer",
    "Jonkheer.": "Royalty",
    "Don.": "Royalty",
    "Sir.": "Royalty",
    "Countess.": "Royalty",
    "Lady.": "Royalty",
    "Mr.": "Mr",
    "Mrs.": "Mrs",
    "Ms.": "Mrs",
    "Mme.": "Mrs",
    "Miss.": "Miss", 
    "Mlle.": "Miss",
    "Master.": "Master",
    "Dona.": "Mrs",
}
def substituir_titulos(linha):
    return titulos[linha]
    
df["Title"] = df["Prefix"].apply(substituir_titulos)
df["Title"].value_counts()

---
## Exercícios

### E4.1.1.
Encontre um passageiro que contenha _Jack_ em seu nome, e outro que contenha *Rose*:

### E4.1.2.
Encontre os passageiros dos quais o `Ticket` contêm a string *CA*. Atribua o novo `DataFrame` à variável `df_ticket_ca`:

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe
df_ticket_ca.head()

### E4.1.3.
Crie uma `Series` da coluna `Name` de `df_ticket_ca` e a atribua à variável `sr_name_ca`:

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe
sr_name_ca.head()

### E4.1.4.
Selecione apenas o sobrenome de `sr_name_ca` e o atribua à variável `sr_surname_ca`:
  
    Dica: regex: (^[^,]+)
        ()   : Grupo de seleção
        ^    : Início da linha
        [^,] : Caracteres que não são vírgula
        +    : 1 ou mais caracteres

    Não esqueca de selecionar o match 0.

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe
sr_surname_ca.head()

### E4.1.5.
Conte os sobrenomes em `sr_surname_ca`:

## 4.2. Contando Tickets
Muitos tickets têm o mesmo número, então podemos contá-los para incluir uma característica no nosso dataset.
![Ticket](dados/Titanic-auction-brochure-ticket-944182.jpg)

Podemos usar o método `value_counts` para contar os tickets, mas também podemos usar um método mais poderoso, o `groupby`.
    
Este método é muito semelhante ao `GROUP BY` do `SQL` e podemos fazer várias agregações com ele. No nosso exemplo, vamos agrupar os dados pela coluna `Ticket` e contar a quantidade de nomes (coluna `Name`).

In [ ]:
df_ticket = df.groupby(["Ticket"])[["Name"]].count()
df_ticket.sort_values("Name", ascending=False).head(10)

Podemos renomear a coluna de um `DataFrame` com o método `rename`:

In [ ]:
df_ticket = df_ticket.rename(columns={"Name": "TicketCount"})
df_ticket.head()

Para inserir as informações da contagem no `DataFrame` `df` podemos usar o método `merge` (`JOIN` no `SQL`). Vamos unir o dataframe `df` na coluna `Ticket` com o `df_ticket` em seu índice:

In [ ]:
df = pd.merge(df, df_ticket, left_on="Ticket", right_index=True).sort_index()
df.head()

---
## Exercícios
### E4.2.1.
Conte os _nomes_ (`Name`) dos passageiros, agrupando-os por _sexo_ (`Sex`)e _idade_ (`Age`).

### E4.2.2.
Crie uma coluna _booleana_ no `DataFrame` `df` chamada `Child` na qual o valor é `True` quando o passageiro tem menos de 16 anos.

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe
df[df["Child"]].head()

### E4.2.3.
Conte os nomes (`Name`) dos passageiros agrupando os dados pelas colunas `Sex`, `Pclass`, e `Child`. Armazene esses dados no dataframe `df_populacoes_count`.

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe
df_populacoes_count

### E4.2.4.
Some a coluna `Survived` dos passageiros, agrupando os dados pelas colunas `Sex`, `Pclass`, e `Child`. Armazene esses dados no dataframe `df_populacoes_survived`.

Dica: Para somar, troque o `count` por `sum`.

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe
df_populacoes_survived

### E4.2.4.
Divida a coluna `Survived` de `df_populacoes_survived` pela coluna `Name` de `df_populacoes_count`. Armazene esses dados em `df_populacoes_rate`:

In [ ]:
df_populacoes_rate =

# O código abaixo é para plotar, não altere.
fig, ax = plt.subplots(1, 2, figsize=(15, 6))
pd.DataFrame(df_populacoes_rate.loc["male"]*100).plot.bar(figsize=(15, 8), rot=30, ax=ax[0], color=PALETA[0])
pd.DataFrame(df_populacoes_rate.loc["female"]*100).plot.bar(figsize=(15, 8), rot=30, ax=ax[1], color=PALETA[1])
ax[0].set_title("Taxa de sobrevivência - Sexo masculino")
ax[1].set_title("Taxa de sobrevivência - Sexo feminino")
ax[0].set_ylabel("%")
ax[1].set_ylabel("%")
ax[0].legend().remove()
ax[1].legend().remove()

O agrupamento é uma ferramenta poderosa. Podemos resumir todos os passos que fizemos anteriormente em apenas uma linha:

In [ ]:
df.groupby(["Sex", "Pclass", "Child"]).agg({"Name": "count", "Survived": ["sum", "mean"]})

## 4.3. Operações em colunas
O `pandas` permite fazer diversas operações numéricas de maneira muito simples e intuitiva:
* `+` &nbsp;&nbsp;$\rightarrow$ `__add__`
* `-` &nbsp;&nbsp;$\rightarrow$ `__sub__`
* `*` &nbsp;&nbsp;$\rightarrow$ `__mul__`
* `/` &nbsp;&nbsp;$\rightarrow$ `__div__`
* `//` $\rightarrow$ `__floordiv__`
* `**` $\rightarrow$ `__pow__`
* `%` &nbsp;&nbsp;$\rightarrow$ `__mod__`
* `|` &nbsp;&nbsp;$\rightarrow$ `__or__`
* `&` &nbsp;&nbsp;$\rightarrow$ `__and__`
* `^` &nbsp;&nbsp;$\rightarrow$ `__xor__`
 
Podemos determinar a quantidade de parentes por passageiro, somando as características `SibSp` (Irmãos/Esposo(a)s) e `Parch` (Pais/Filhos) + 1, que é o próprio passageiro. Vamos armazenar essa informação numa nova coluna `FamilySize`:

In [ ]:
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
df.head()

---
## Exercícios
### E4.3.1.
Vamos criar mais uma característica. Estimaremos o custo da tarifa pago por família. A estimativa é o valor da tarifa (`Fare`) multiplicada pelo tamanho da família (`FamilySize`). Armazene essa informação na coluna `FamilyFare`:

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe
df[["FamilyFare"]].head()

### E4.3.2.
Vamos criar uma característica para indicar as pessoas que estão viajando sem família. Crie uma coluna booleana chamada `NoRelatives` que é `True` quando o tamanho da família é 1 (`FamilySize`):

In [ ]:

# A linha abaixo serve somente para mostrar o dataframe
df["NoRelatives"].head()

Conseguimos extrair mais seis caracterísitcas dos dados. Será que conseguimos melhorar o modelo?

# 5. Mais árvores
![duas arvores](dados/two_trees_and_flock.jpg)

Vamos montar mais uma árvore de decisão. Dessa vez utilizaremos as seguintes características:
* Características binárias:
  * `Sex`
  * `NoRelatives`
  * `Young`
* Caracterísitcas categóricas:
  * `PClass`
  * `Embarked`
  * `Title`
* Características numéricas
  * `Age`
  * `SibSp`
  * `Parch`
  * `Fare`
  * `TicketCount`
  * `FamilySize`
  * `FamilyFare`

In [ ]:
colunas_binarias = ["Sex", "NoRelatives", "Child"]
colunas_categoricas = ["Pclass", "Embarked", "Title"]
colunas_numericas = ["Age", "SibSp", "Parch", "Fare", "TicketCount", "FamilySize", "FamilyFare"]

def processar_dataframe(df, colunas_binarias, colunas_categoricas, colunas_numericas):
    df_saida = pd.get_dummies(df[colunas_categoricas].astype(str))
    for col in colunas_binarias:
        if df[col].dtype == bool:
            df_saida[col] = df[col]
        else:
            sr_bin = df[col]
            prefixo = df[col][0]
            df_saida[col+"_"+prefixo] = sr_bin == prefixo
    df_saida = pd.concat([df_saida, df[colunas_numericas]], axis=1)
    return df_saida


X2 = processar_dataframe(df, colunas_binarias, colunas_categoricas, colunas_numericas)
Y2 = df["Survived"]
X2.head()

Notem que transformamos as colunas categóricas em colunas numéricas usando o _One hot encoding_ , ou seja, criamos uma coluna de característica para cada categoria.

Modelando os dados na árvore de decisão temos a seguinte acurácia:

In [ ]:
# 1. Instanciando o modelo
modelo_arvore2 = tree.DecisionTreeClassifier(max_depth=4, min_samples_leaf=5)

# 2. Treinando o modelo
modelo_arvore2.fit(X2, Y2)

# 3. Gerando as predições
df["predicao arvore 2"] = modelo_arvore2.predict(X2)

acc_arvore2 = metrics.accuracy_score(
    df["Survived"], 
    df["predicao arvore 2"]
)
"Acurácia: {}%".format(100*acc_arvore2)

In [ ]:
def plot_acuracias(nomes_modelos, acuracias, paleta, titulo, rot=0):
    fig, ax = plt.subplots(figsize=(15, 6))
    ax.set_ylim(0, 100)
    ax.set_title(titulo)
    ax.set_ylabel("%")
    plt.xticks(rotation=rot)
    
    rects = ax.bar(
        nomes_modelos,
        acuracias,
        color=paleta,
    )
    for rect in rects:
        height = rect.get_height()
        ax.annotate("{:.3f}".format(float(height)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    ha="center", va='bottom')
plot_acuracias(
    ["Simples", "Randômico", "Árvore 1", "Árvore 2"], 
    [100*acc_simples, 100*acc_randomica, 100*acc_arvore1, 100*acc_arvore2],
    PALETA,
    "Acurácia Treino"
)

In [ ]:
plot_tree(modelo_arvore2, X2.columns, classes)

Algumas conclusões que podemos tomar com base nessa árvore:
* Cerca de 15.6% dos passageiros do sexo masculino (`Title` $=$ *Mr*) sobreviveram. Esses resultados são similares aos encontrados no final do capítulo 2.
* 94% Dos passageiros que não pertencem às categorias `Title` $=$ *Mr e Officer* têm uma família com *4* pessoas ou menos e `PClass` $\neq$ *3* sobreviveram.
* Entre os homens (`Title` $=$ *Mr*), os passageiros da primeira classe (`PClass` $\neq$ *2* e `Fare` $>$ *26.775*) foram os que mais sobreviveram (taxa de 31.6%, o dobro da taxa para homens).

---
## Exercícios
### E5.
Altere as características selecionadas para a modelagem e tente interpretar as árvores de decisão

In [ ]:
## Altere as características de interesse ou até crie novas características.

colunas_binarias_e5 = ["Sex", "NoRelatives", "Child"]
colunas_categoricas_e5 = ["Pclass", "Embarked", "Title"]
colunas_numericas_e5 = ["Age", "SibSp", "Parch", "Fare", "FamilySize"]
colunas_descartadas = ["TicketCount", "FamilyFare"]

profundidade = 8  # Profundidade da árvore
minimo_amostras_por_folha = 5 # Quantidade mínima de dados por folha

## --------------------------------------------------------------------- ##
## Não altere o código abaixo
XE5 = processar_dataframe(df, colunas_binarias_e5, colunas_categoricas_e5, colunas_numericas_e5)
YE5 = df["Survived"]

modelo_arvoree5 = tree.DecisionTreeClassifier(max_depth=profundidade, min_samples_leaf=minimo_amostras_por_folha)
modelo_arvoree5.fit(XE5, YE5)
df["predicao arvore e5"] = modelo_arvoree5.predict(XE5)

acc_arvoree5 = metrics.accuracy_score(
    df["Survived"], 
    df["predicao arvore e5"]
)

plot_acuracias(
    ["Simples", "Randômico", "Árvore 1", "Árvore 2", "Árvore E5"], 
    [100*acc_simples, 100*acc_randomica, 100*acc_arvore1, 100*acc_arvore2, 100*acc_arvoree5],
    PALETA,
    "Acurácia Treino"
)

plot_tree(modelo_arvoree5, XE5.columns, classes)

# 6. Teste de resistência
![Afundando](dados/Stöwer_Titanic.jpg)


Nós conseguimos modelar os dados que tínhamos em mãos, mas será que esse conhecimento generaliza para o conjunto de testes?

Vamos tentar gerar, portanto, a coluna `Survived` para o conjunto de testes `df_teste`. Primeiro precisamos aplicar as mesmas transformações que fizemos para o _conjunto de treino_ `df`.

In [ ]:
df_teste = pd.read_csv(ARQUIVO_TESTE)

df_teste["Age"] = df_teste["Age"].fillna(df["Age"].median())
df_teste["Fare"] = df_teste["Fare"].fillna(df["Fare"].median())

df_teste["Prefix"] = df_teste["Name"].str.extract(expressao_titulo)
df_teste["Title"] = df_teste["Prefix"].apply(substituir_titulos)

df_ticket_teste = df_teste.groupby(["Ticket"])[["Name"]].count()
df_ticket_teste = df_ticket_teste.rename(columns={"Name": "TicketCount"})
df_teste = pd.merge(df_teste, df_ticket_teste, left_on="Ticket", right_index=True).sort_index()

df_teste["Child"] = df_teste["Age"] < 16

df_teste["FamilySize"] = df_teste["SibSp"] + df_teste["SibSp"]
df_teste["FamilyFare"] = df_teste["FamilySize"] * df_teste["Fare"]
df_teste["NoRelatives"] = df_teste["FamilySize"] == 0

X_teste1 = df_teste[["Age", "SibSp", "Parch", "Fare"]]

X_teste2 = processar_dataframe(df_teste, colunas_binarias, colunas_categoricas, colunas_numericas)
X_teste2["Title_Royalty"] = 0
X_teste2 = X_teste2[X2.columns]
X_teste2.head()

Em seguida, com os mesmos modelos, podemos gerar predições. Vamos armazenar as predições nas mesmas colunas criadas anteriormente. 

Finalmente, podemos enviar os dados para o _Kaggle_ para testar o modelo e ver qual a taxa de acerto. Para simplificar esse processo, estamos fornecendo a função `verificar_respostas` que recebe um `DataFrame` e a coluna das respostas e retorna a acurácia do modelo.

In [ ]:
from respostas import verificar_respostas
random.seed(1912)
df_teste["predicao simples"] = 1
df_teste["predicao randomica"] = [random.randint(0, 2) for i in range(len(df_teste))]
df_teste["predicao arvore 1"] = modelo_arvore1.predict(X_teste1)
df_teste["predicao arvore 2"] = modelo_arvore2.predict(X_teste2)

acc_teste_simples = verificar_respostas(df_teste, "predicao simples")
acc_teste_randomica = verificar_respostas(df_teste, "predicao randomica")
acc_teste_arvore1 = verificar_respostas(df_teste, "predicao arvore 1")
acc_teste_arvore2 = verificar_respostas(df_teste, "predicao arvore 2")

plot_acuracias(
    ["Simples Treino", "Simples Teste", "Randômico Treino", "Randômico Teste", 
     "Árvore 1 Treino", "Árvore 1 Teste", "Árvore 2 Treino", "Árvore 2 Teste"], 
    [100*acc_simples, 100*acc_teste_simples, 100*acc_randomica, 100*acc_teste_randomica, 
     100*acc_arvore1, 100*acc_teste_arvore1, 100*acc_arvore2, 100*acc_teste_arvore2],
    [PALETA[0]]*2 + [PALETA[1]]*2+[PALETA[2]]*2 + [PALETA[3]]*2,
    "Acurácia Treino e Teste"
)

Percebemos que a acurácia no _conjunto de teste_ para os modelos de árvore são sempre menores. Isso se deve ao fato de que o conhecimento modelado no _conjunto de treino_ nem sempre é o que melhor representa a realidade.

Para algumas arquiteturas de modelo, é possível modelar o _conjunto de treino_ perfeitamente, ou seja, obter uma acurácia de 100%. Etretanto, esse valor não representa a distribuição total dos dados, e a acurácia no conjunto de testes pode ser até pior que a de alguns modelos mais simples.

---
## Exercícios
### E6.
Veja a acurácia do modelo criado no **exercício 5**, não é preciso editar o código abaixo. Se desejar, continue alterando o modelo **E5** e veja a diferença na acurácia de teste aqui:

In [ ]:
X_testeE6 = processar_dataframe(df_teste, colunas_binarias_e5, colunas_categoricas_e5, colunas_numericas_e5)
X_testeE6["Title_Royalty"] = 0
X_testeE6 = X_testeE6[XE5.columns]
df_teste["predicao arvore e5"] = modelo_arvoree5.predict(X_testeE6)

acc_teste_arvoree5 = verificar_respostas(df_teste, "predicao arvore e5")

plot_acuracias(
    ["Simples Treino", "Simples Teste", "Randômico Treino", "Randômico Teste", "Árvore 1 Treino", 
     "Árvore 1 Teste", "Árvore 2 Treino", "Árvore 2 Teste", "Árvore E5 Treino", "Árvore E5 Teste"], 
    [100*acc_simples, 100*acc_teste_simples, 100*acc_randomica, 100*acc_teste_randomica, 100*acc_arvore1, 
     100*acc_teste_arvore1, 100*acc_arvore2, 100*acc_teste_arvore2, 100*acc_arvoree5, 100*acc_teste_arvoree5],
    [PALETA[0]]*2 + [PALETA[1]]*2+[PALETA[2]]*2 + [PALETA[3]]*2 + [PALETA[4]]*2,
    "Acurácia Treino e Teste",
    rot=15
)

# Afundando!
<img src="dados/jack.gif" width="800px">

<span style="font-size: 2em">Phew!</span> Lá se foi mais um bocado de tópicos!

1. Implementamos um modelo de árvore de decisão e usamos a acurácia como métrica para avaliar a qualidade dos modelos.
2. Trabalhamos com métodos para strings e vimos o poder do `regex`.
3. Fizemos agrupamentos e fundimos tabelas de dados.
4. Fizemos operações matemáticas nas colunas de dados.
5. Treinamos outra árvore de decisão com mais caracterísitcas.
6. Avaliamos os modelos no conjunto de dados de testes 

Enfim, `pandas` é vento em popa pra análise de dados!
## Muito Obrigado!